# DataUp

Proyecto del curso "Taller de Proyecto". Análisis de entrevistas utilizando procesamiento de voz y técnicas de análisis de texto

In [9]:
#Imports
import speech_recognition as sr #Speech Recognition
from os import path
import matplotlib.pyplot as plt

## 1.- Procesamiento de Voz

## 2.- Análisis del Texto

In [24]:
audioFile = path.join(path.dirname(path.realpath("bodoque.wav")), "bodoque.wav")

reconocedor = sr.Recognizer()
with sr.AudioFile(audioFile) as source:
    audio = reconocedor.record(source)
    
# recognize speech using Google Speech Recognition
try:
    # for testing purposes, we're just using the default API key
    # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    # instead of `r.recognize_google(audio)`
    audioTranscrito = r.recognize_google(audio, language = "es-CL")
    print("Google Speech Recognition thinks you said " + audioTranscrito)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))


Google Speech Recognition thinks you said la muerte de mi tía Eulalia me dejó como el único heredero eso fértiles tierras cerca de Valle la noticia me vino como anillo al dedo porque tenía que pagar unas deudas contraídas por mi afición al juego vendería los terrenos y quedaría libre de tan desagradable
